## *Análise da alavancagem do não-circulante*

In [2]:
# bibliotecas usadas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [3]:
demonstracoes = pd.read_excel('D:\{estudos}\Python\ADC - Embraer S.A\Analise-Embraer\Demonstrações_Embraer_Ajustado.xlsm', sheet_name=None)
demonstracoes.keys()

<>:1: SyntaxWarning: invalid escape sequence '\{'
<>:1: SyntaxWarning: invalid escape sequence '\{'
C:\Users\Madu\AppData\Local\Temp\ipykernel_28316\71981632.py:1: SyntaxWarning: invalid escape sequence '\{'
  demonstracoes = pd.read_excel('D:\{estudos}\Python\ADC - Embraer S.A\Analise-Embraer\Demonstrações_Embraer_Ajustado.xlsm', sheet_name=None)


dict_keys(['Ativo', 'Passivo', 'DRE', 'DFC'])

In [4]:
# Extração dos dados da planilha
if demonstracoes:
    df_ativo = demonstracoes.get('Ativo')
    df_passivo = demonstracoes.get('Passivo')
    df_dre = demonstracoes.get('DRE')

    # verificada manualmente a correspondência dos códigos com os demonstrativos
    contas_chave = {
        "AtivoCirculante": (df_ativo, "1.01"),
        "AtivoNaoCirculante": (df_ativo, "1.02"),
        "AtivoImobilizado": (df_ativo, "1.02.03"),
        "AtivoRealizavelLP": (df_ativo, "1.02.01"),
        "AtivoCirculante":      (df_ativo, "1.01"),
        "AtivoTotal": (df_ativo, "1"),
        "PassivoTotal": (df_passivo, "2"),
        "PassivoCirculante": (df_passivo, "2.01"),
        "PassivoNaoCirculante": (df_passivo, "2.02"),
        "PatrimonioLiquido": (df_passivo, "2.03"),
        "ReceitaLiquida": (df_dre, "3.01"),
        "LucroBruto": (df_dre, "3.03"),
        "LucroOperacional": (df_dre, "3.05"),
        "LucroLiquido": (df_dre, "3.11"),
        "AtivoPermanente": (df_ativo, "1.02")
    }
    
    # Extrai os dados para um DataFrame organizado por ano
    anos = [col for col in df_ativo.columns if isinstance(col, (int, float))]
    base_data = pd.DataFrame(index=contas_chave.keys(), columns=anos)

    print("\n--- Extraindo valores-chave dos demonstrativos ---")
    for nome, (df, codigo) in contas_chave.items():
        if df is not None:
            # A busca agora ocorre no dataframe com colunas já limpas
            linha = df[df['CD_CONTA'] == str(codigo)] # Garante que o código seja string
            if not linha.empty:
                valores = linha.iloc[0][anos]
                base_data.loc[nome] = valores
            else:
                print(f"   AVISO: Conta '{nome}' (código {codigo}) não encontrada. Será preenchida com 0.")
                base_data.loc[nome] = 0

    print("\n--- Base de Dados para Cálculo ---")
    print(base_data.to_string())


--- Extraindo valores-chave dos demonstrativos ---

--- Base de Dados para Cálculo ---
                                 2020             2021             2022             2023        2024
AtivoCirculante       40549746.680452  37967991.536853  32876200.424879     30670519.513    40326906
AtivoNaoCirculante    29141300.635077  27650888.976054  25044728.794734    23942405.2206    32892812
AtivoImobilizado      13366071.253672  11293848.893319   9790271.260124     9414354.1516    12669526
AtivoRealizavelLP      1995054.146193   2026121.285432   2354678.507979      2578961.465     4454043
AtivoTotal            69691047.315529  65618880.512908  57920929.219613    54612924.7336    73219718
PassivoTotal          69691047.315529  65618880.512908  57920929.219613    54612924.7336  73219718.0
PassivoCirculante      16084627.91635  18279637.446943  18373669.443463    18565945.7574  27347016.0
PassivoNaoCirculante  34305057.823225  29409556.675324  23417345.436912    20652951.5502  25161794.0
Pat

### Calculando os indicadores do não circulante

In [5]:
indicadores = pd.DataFrame(columns=anos) # DataFrame para os resultados

# =============================================================================
# INDICADORES DE ESTRUTURA DE CAPITAL (NÃO CIRCULANTE)

# 1. Endividamento Total
indicadores.loc['Endividamento Total'] = base_data.loc['PassivoTotal'] / base_data.loc['PatrimonioLiquido']

# 2. Dependência/Independência Financeira
indicadores.loc['Dependência Financeira (%)'] = (base_data.loc['PassivoTotal'] / base_data.loc['AtivoTotal']) * 100
indicadores.loc['Independência Financeira (%)'] = (base_data.loc['PatrimonioLiquido'] / base_data.loc['AtivoTotal']) * 100

# 3. Participação de Capital de Terceiros
indicadores.loc['Capital Terceiros LP'] = (base_data.loc['PassivoCirculante'] + base_data.loc['PassivoNaoCirculante']) / base_data.loc['PatrimonioLiquido']

# =============================================================================
# INDICADORES DE IMOBILIZAÇÃO

# 4. Imobilização do Capital Permanente
indicadores.loc['Imobilização Capital Permanente'] = base_data.loc['AtivoNaoCirculante'] / (base_data.loc['PatrimonioLiquido'] + base_data.loc['PassivoNaoCirculante'])

# 5. Imobilização do Patrimônio Líquido
indicadores.loc['Imobilização PL'] = base_data.loc['AtivoPermanente'] / base_data.loc['PatrimonioLiquido']

# 6. Imobilização de Recursos Não Correntes
indicadores.loc['Imobilização Recursos Não Correntes'] = (
    (base_data.loc['AtivoNaoCirculante'] - base_data.loc['AtivoRealizavelLP']) / 
    (base_data.loc['PassivoNaoCirculante'] + base_data.loc['PatrimonioLiquido'])
)


# =============================================================================
# INDICADORES DE DESEMPENHO DO IMOBILIZADO

# 7. Giro do Imobilizado
indicadores.loc['Giro do Imobilizado'] = base_data.loc['ReceitaLiquida'] / base_data.loc['AtivoImobilizado']

# =============================================================================
# INDICADORES DE LIQUIDEZ E COMPOSIÇÃO

# 9. Composição do Endividamento
indicadores.loc['Composição Endividamento (%)'] = (
    (base_data.loc['PassivoNaoCirculante'] + base_data.loc['PassivoCirculante']) / base_data.loc['PassivoTotal']
) * 100

# 9. Liquidez Geral
indicadores.loc['Liquidez Geral'] = (
    (base_data.loc['AtivoCirculante'] + base_data.loc['AtivoRealizavelLP']) / 
    (base_data.loc['PassivoCirculante'] + base_data.loc['PassivoNaoCirculante'])
)

# 10. Composição do Endividamento (Longo Prazo)
indicadores.loc['Composição Endividamento LP (%)'] = (
    base_data.loc['PassivoNaoCirculante'] / base_data.loc['PassivoTotal']
) * 100


# =============================================================================
# INDICADORES ADICIONAIS RELEVANTES

# Participação do Não Circulante no Ativo Total
indicadores.loc['Participação ANC no Ativo (%)'] = (
    base_data.loc['AtivoNaoCirculante'] / base_data.loc['AtivoTotal']
) * 100

# 2. CÁLCULO DOS INDICADORES DE RENTABILIDADE
# --- Margens ---
# Margem Operacional = EBIT (3.05) / Receita Líquida
indicadores.loc['Margem Operacional (%)'] = (base_data.loc['LucroOperacional'] / base_data.loc['ReceitaLiquida']) * 100

# Margem Líquida = Lucro Líquido (3.11) / Receita Líquida
indicadores.loc['Margem Líquida (%)'] = (base_data.loc['LucroLiquido'] / base_data.loc['ReceitaLiquida']) * 100

# --- Retornos (Rentabilidade) ---
# ROA (Operacional) = EBIT (3.05) / Ativo Total
# Nota: Usando 3.05 conforme solicitado (foco na eficiência operacional dos ativos)
indicadores.loc['ROA (%)'] = (base_data.loc['LucroOperacional'] / base_data.loc['AtivoTotal']) * 100

# ROE = Lucro Líquido (3.11) / Patrimônio Líquido
indicadores.loc['ROE (%)'] = (base_data.loc['LucroLiquido'] / base_data.loc['PatrimonioLiquido']) * 100

print("\n\n--- INDICADORES DE ANÁLISE DO NÃO CIRCULANTE ---")
print("=" * 60)


print("\n\n--- Tabela Final de Indicadores Financeiros ---")
print(indicadores.round(2).to_string())



--- INDICADORES DE ANÁLISE DO NÃO CIRCULANTE ---


--- Tabela Final de Indicadores Financeiros ---
                                          2020       2021       2022       2023       2024
Endividamento Total                    3.61068    3.65979   3.590901    3.54767   3.535321
Dependência Financeira (%)               100.0      100.0      100.0      100.0      100.0
Independência Financeira (%)         27.695611  27.323975  27.848162  28.187517  28.285971
Capital Terceiros LP                   2.61068    2.65979   2.590901    2.54767   2.535321
Imobilização Capital Permanente       0.543616   0.584101   0.633286     0.6642   0.717045
Imobilização PL                       1.509805   1.542185   1.552688   1.555305   1.588188
Imobilização Recursos Não Correntes   0.506399   0.541301   0.573745   0.592656    0.61995
Giro do Imobilizado                    1.70785   2.214638   2.517534   2.912558   2.796014
Composição Endividamento (%)         72.304389  72.676025  72.151838  71.812483 

### Cálculo de alavancagem

In [14]:
# --- CÁLCULO DAS ALAVANCAGENS (MÉTODO DA VARIAÇÃO) ---

# Primeiro, calculamos a variação percentual de um ano para o outro
# Transpomos (.T) para calcular a variação por coluna (ano) e transpomos de volta
variacoes = base_data.T.pct_change().T

# Iteramos sobre as colunas de anos (pulando o primeiro ano, pois não tem anterior)
colunas_anos = base_data.columns

for i in range(0, len(colunas_anos)):
    ano_atual = colunas_anos[i]
    
    # Buscamos as variações já calculadas
    var_receita = variacoes.loc['ReceitaLiquida', ano_atual]
    var_ebit = variacoes.loc['LucroOperacional', ano_atual] # EBIT
    var_lucro_liq = variacoes.loc['LucroLiquido', ano_atual]

    # 1. GAO - Grau de Alavancagem Operacional
    # Fórmula: Variação % do EBIT / Variação % da Receita
    # Interpretação: Para cada 1% de aumento na receita, quanto varia o lucro operacional?
    if var_receita != 0:
        gao = var_ebit / var_receita
    else:
        gao = 0 

    # 2. GAF - Grau de Alavancagem Financeira
    # Fórmula: Variação % do Lucro Líquido / Variação % do EBIT
    # Interpretação: Para cada 1% de variação no operacional, quanto impacta o lucro líquido?
    if var_ebit != 0:
        gaf = var_lucro_liq / var_ebit
    else:
        gaf = 0

    # 3. GAT - Grau de Alavancagem Total
    # Fórmula: GAO * GAF
    gat = gao * gaf

    # Adicionando ao DataFrame de indicadores
    # O primeiro ano ficará vazio/NaN ou você pode preencher com 0 depois
    indicadores.loc['GAO (Alavancagem Operacional)', ano_atual] = gao
    indicadores.loc['GAF (Alavancagem Financeira)', ano_atual] = gaf
    indicadores.loc['GAT (Alavancagem Total)', ano_atual] = gat

    # Opcional: Adicionar as variações para facilitar a análise
    indicadores.loc['Var. % Receita', ano_atual] = var_receita * 100
    indicadores.loc['Var. % EBIT', ano_atual] = var_ebit * 100
    indicadores.loc['Var. % Lucro Líquido', ano_atual] = var_lucro_liq * 100

print("\n--- Indicadores de Alavancagem Calculados ---")
# Filtra apenas as linhas de alavancagem para visualizar
cols_alavancagem = [col for col in indicadores.index if 'Alavancagem' in col]


print(indicadores.loc[cols_alavancagem].to_string())



--- Indicadores de Alavancagem Calculados ---
                              2020       2021        2022       2023      2024
GAO (Alavancagem Operacional)  NaN -16.673436  103.070286  -33.23053  4.655799
GAF (Alavancagem Financeira)   NaN   0.582437   -1.816406   0.467709  0.982729
GAT (Alavancagem Total)        NaN  -9.711225 -187.217496 -15.542229  4.575386


C:\Users\Madu\AppData\Local\Temp\ipykernel_28316\3374499568.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  variacoes = base_data.T.pct_change().T


In [15]:
# Formata a saída para melhor visualização
for indicador in indicadores.index:
    print(f"\n{indicador}:")
    for ano in indicadores.columns:
        valor = indicadores.loc[indicador, ano]
        if any(termo in indicador for termo in ['%', 'Percent', 'Composição', 'Dependência', 'Independência', 'Participação']):
            print(f"  {ano}: {valor:.2f}%")
        elif 'Vida Útil' in indicador:
            print(f"  {ano}: {valor:.1f} anos")
        else:
            print(f"  {ano}: {valor:.4f}")

# Tabela resumo
print("\n\n--- TABELA RESUMO DOS INDICADORES ---")
print(indicadores.round(4).to_string())


Endividamento Total:
  2020: 3.6107
  2021: 3.6598
  2022: 3.5909
  2023: 3.5477
  2024: 3.5353

Dependência Financeira (%):
  2020: 100.00%
  2021: 100.00%
  2022: 100.00%
  2023: 100.00%
  2024: 100.00%

Independência Financeira (%):
  2020: 27.70%
  2021: 27.32%
  2022: 27.85%
  2023: 28.19%
  2024: 28.29%

Capital Terceiros LP:
  2020: 2.6107
  2021: 2.6598
  2022: 2.5909
  2023: 2.5477
  2024: 2.5353

Imobilização Capital Permanente:
  2020: 0.5436
  2021: 0.5841
  2022: 0.6333
  2023: 0.6642
  2024: 0.7170

Imobilização PL:
  2020: 1.5098
  2021: 1.5422
  2022: 1.5527
  2023: 1.5553
  2024: 1.5882

Imobilização Recursos Não Correntes:
  2020: 0.5064
  2021: 0.5413
  2022: 0.5737
  2023: 0.5927
  2024: 0.6199

Giro do Imobilizado:
  2020: 1.7079
  2021: 2.2146
  2022: 2.5175
  2023: 2.9126
  2024: 2.7960

Composição Endividamento (%):
  2020: 72.30%
  2021: 72.68%
  2022: 72.15%
  2023: 71.81%
  2024: 71.71%

Liquidez Geral:
  2020: 0.8443
  2021: 0.8386
  2022: 0.8430
  2023: 0.

### Exportar a tabela para excel

In [16]:
nome_arquivo_saida = "Indicadores_Financeiros_Embraer.xlsx"
try:
    # Cria um "escritor" de Excel para salvar múltiplas abas
    with pd.ExcelWriter(nome_arquivo_saida, engine='openpyxl') as writer:
        indicadores.to_excel(writer, sheet_name='Indicadores')
        base_data.to_excel(writer, sheet_name='Dados Base')
            
    print(f"\n\n Tabela de indicadores exportada com sucesso para o arquivo: '{nome_arquivo_saida}'")
    print("   O arquivo contém duas abas: 'Indicadores' e 'Dados Base'.")
except Exception as e:
    print(f"\n ERRO ao exportar para Excel: {e}")



 Tabela de indicadores exportada com sucesso para o arquivo: 'Indicadores_Financeiros_Embraer.xlsx'
   O arquivo contém duas abas: 'Indicadores' e 'Dados Base'.


### Análise dos indicadores do não circulante

### `Indicadores Consistentes`

### - Estrutura de Capital
- **Endividamento Total (~2.5-2.6)**: Muito alto - para cada R$1 de PL, há R$2,5-2,6 de dívida
- **Dependência Financeira (~72%)**: 72% do ativo é financiado por terceiros (preocupante)
- **Independência Financeira (~28%)**: Apenas 28% com recursos próprios (baixo)

### - Composição do Endividamento
- **Composição LP (%)**: Caiu de 68% para 48% - migrando dívidas para curto prazo (risco)

### - Participação ANC
- **~42-45% do Ativo Total**: Valor razoável para empresa industrial

### `Indicadores com Possíveis Problemas`

### - Liquidez Geral (~0.84-0.85)
**PROBLEMA GRAVE**: Valor abaixo de 1.0 indica que:
- Ativos de curto e longo prazo NÃO cobrem obrigações totais
- Risco de insolvência se credores exigirem pagamento

### - Giro do Imobilizado (1.7-2.9)
Valores baixos para empresa industrial. Ideal seria > 3.0

### - Capital Terceiros LP (1.2-1.8)
Muito elevado - dívida de LP maior que o PL